In [1]:
cd("G:\\My Drive\\_MOPJ\\Julia\\ROM4SM\\")

In [2]:
readdir()

6-element Array{String,1}:
 ".git"
 "LICENSE"
 "POD4Frame.ipynb"
 "POD4Frame.jl"
 "POD4Vectors.jl"
 "img"

# Treillis
<img src="https://raw.githubusercontent.com/amdeld/ROM4SM.jl/master/img/TrussStructure_LBC.png" alt="Drawing" style="width: 1000px;"/>


# Fonction Matrice de raideur élémentaire pour un élément de barre plan

In [3]:
function trstiff(coo,E,A)
# Function that computes stiffness matrix for truss element
# (file name trstiff.m)
L=sqrt((coo[2,1]-coo[1,1])^2+(coo[2,2]-coo[1,2])^2)
C=(coo[2,1]-coo[1,1])/L
S=(coo[2,2]-coo[1,2])/L
# Stiffness matrix
STF=E*A/L*[C^2 S*C -C^2 -S*C;
S*C S^2 -S*C -S^2;
-C^2 -S*C C^2 S*C;
-S*C -S^2 S*C S^2];
return(STF)
end

trstiff (generic function with 1 method)

# Fonction d'assemblage

In [4]:
# Main program for truss structure
# Coordinates - units L[mm]
function truss_str(E,A)
COO=[0 0; 2 0; 4 0 ;1 1; 3 1; -1 1; 5 1]
COO=1000*COO
DOF=size(COO,1)*2
# Elements
ELM=[1 2; 2 3; 6 4; 4 5; 5 7; 1 6; 1 4; 2 4; 2 5; 3 5; 3 7]
# Properties (units F[N], L[mm])
#E=125000
#A=6
# Assembling of stiffness matrix
MSTF=zeros(DOF,DOF)
X=zeros(2,2)
for i in 1:size(ELM,1)
X[1,:]=COO[ELM[i,1],:]
X[2,:]=COO[ELM[i,2],:]
STF=trstiff(X,E,A)
MSTF[2*ELM[i,1]-1:2*ELM[i,1],2*ELM[i,1]-
1:2*ELM[i,1]]=MSTF[2*ELM[i,1]-1:2*ELM[i,1],2*ELM[i,1]-
1:2*ELM[i,1]]+STF[1:2,1:2]
MSTF[2*ELM[i,1]-1:2*ELM[i,1],2*ELM[i,2]-
1:2*ELM[i,2]]=MSTF[2*ELM[i,1]-1:2*ELM[i,1],2*ELM[i,2]-
1:2*ELM[i,2]]+STF[3:4,1:2]
MSTF[2*ELM[i,2]-1:2*ELM[i,2],2*ELM[i,1]-
1:2*ELM[i,1]]=MSTF[2*ELM[i,2]-1:2*ELM[i,2],2*ELM[i,1]-
1:2*ELM[i,1]]+STF[1:2,3:4]
MSTF[2*ELM[i,2]-1:2*ELM[i,2],2*ELM[i,2]-
1:2*ELM[i,2]]=MSTF[2*ELM[i,2]-1:2*ELM[i,2],2*ELM[i,2]-
1:2*ELM[i,2]]+STF[3:4,3:4]
end
# Constrains
con=[12 13 14 15] # constrained DOF
CDOF=size(con,2)
# Forces
dF=zeros(DOF,1)
dF[2]=-5000
dF[6]=-2000
# Reduced stiffness matrix & force vector
RSTF=MSTF[1:DOF-CDOF,1:DOF-CDOF]
dFR=dF[1:DOF-CDOF]
# Solving for displacements
d=inv(RSTF)*dFR
# End of main program
return(d)
end

truss_str (generic function with 1 method)

# Fonction Plan d'expériences (Design Of Experiments doe)

In [5]:
function doe()
    cnt = 1
    U=zeros(10)
    for E in [125000 150000 175000 200000]
        for A in [6 8 10 12]
            Disp = truss_str(E, A)
            #show(Disp)
            if cnt > 1
                U = [U Disp]
            else
                U = Disp
            end
            cnt += 1
        end
    end
    #show(U)
    return (U)
end

doe (generic function with 1 method)

# Exécution du plan d'expériences
Variation du module de Young E = {125000; 150000; 175000; 200000} (unité MPa)

Variation de la section A = {6; 8; 10; 12} (unité mm)

In [13]:
# Execute The design of experiments (DOE)
U=doe()
println("\nThe Data Matrix U coming out from the DOE\n")
show(stdout,"text/plain", round.(U;digits=2))
#


The Data Matrix U coming out from the DOE

10×16 Array{Float64,2}:
 -11.56   -8.67   -6.93   -5.78   -9.63   -7.22   -5.78   -4.81   -8.25   -6.19   -4.95   -4.13   -7.22   -5.42   -4.33   -3.61
 -28.53  -21.39  -17.12  -14.26  -23.77  -17.83  -14.26  -11.89  -20.38  -15.28  -12.23  -10.19  -17.83  -13.37  -10.7    -8.91
  -0.89   -0.67   -0.53   -0.44   -0.74   -0.56   -0.44   -0.37   -0.63   -0.48   -0.38   -0.32   -0.56   -0.42   -0.33   -0.28
 -31.87  -23.9   -19.12  -15.93  -26.55  -19.92  -15.93  -13.28  -22.76  -17.07  -13.66  -11.38  -19.92  -14.94  -11.95   -9.96
   7.11    5.33    4.27    3.56    5.93    4.44    3.56    2.96    5.08    3.81    3.05    2.54    4.44    3.33    2.67    2.22
 -16.54  -12.4    -9.92   -8.27  -13.78  -10.34   -8.27   -6.89  -11.81   -8.86   -7.09   -5.91  -10.34   -7.75   -6.2    -5.17
  -2.67   -2.0    -1.6    -1.33   -2.22   -1.67   -1.33   -1.11   -1.9    -1.43   -1.14   -0.95   -1.67   -1.25   -1.0    -0.83
 -35.53  -26.65  -21.32  -17.76  -29

# Vérification Résultats 1ère colonne (E=125000 Mpa et A=6mm) 
<img src="https://raw.githubusercontent.com/amdeld/ROM4SM.jl/master/img/TrussStructure_DisplacementX.png" alt="Drawing" style="width: 1000px;"/>
<img src="https://raw.githubusercontent.com/amdeld/ROM4SM.jl/master/img/TrussStructure_DisplacementY.png" alt="Drawing" style="width: 1000px;"/>

In [8]:
using LinearAlgebra
# Proper Orthogonal Decomposition of the Data Matrix U

In [10]:
F=eigen(U*U')
println("\nEigenValues λ\n")
F.values


EigenValues λ



10-element Array{Float64,1}:
    -1.6658361335067054e-13
    -5.462803647822355e-14
    -1.817489391522599e-14
    -1.009819933969701e-14
     4.710919964930092e-16
     3.999910246483101e-13
     5.244053535157974e-13
     5.733961310215929e-13
     1.8189894035458565e-11
 24580.216532550156

In [19]:
println("\n\nEigenVectors Matrix Φ\n")
Φ=F.vectors
show(stdout,"text/plain", round.(Φ;digits=2))



EigenVectors Matrix Φ

10×10 Array{Float64,2}:
 -0.81   0.45   0.22   0.03   0.06  -0.13   0.17   0.01  -0.08  -0.18
  0.33   0.5   -0.3   -0.02  -0.04  -0.5   -0.26   0.04  -0.2   -0.43
 -0.09  -0.03  -0.02   0.13  -0.99   0.01   0.0    0.01  -0.01  -0.01
 -0.06  -0.02  -0.24  -0.14  -0.0    0.58  -0.04  -0.55  -0.23  -0.48
 -0.38  -0.31  -0.63  -0.53  -0.02  -0.2   -0.11   0.14   0.05   0.11
 -0.02  -0.52   0.31  -0.05   0.0   -0.57   0.16  -0.45  -0.12  -0.25
 -0.2   -0.22  -0.45   0.82   0.14  -0.07  -0.02  -0.02  -0.02  -0.04
 -0.02  -0.35   0.17  -0.01   0.03   0.16   0.02   0.69  -0.25  -0.54
  0.2    0.11  -0.27  -0.05  -0.02  -0.02   0.93   0.07  -0.02  -0.04
  0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.9   -0.43

In [20]:
println("\nAmplitude Matrix A\n")
A=F.vectors'*U
show(stdout,"text/plain", round.(A;digits=2))


Amplitude Matrix A

10×16 Array{Float64,2}:
 -0.0    0.0    0.0   -0.0    0.0    0.0   -0.0    0.0    0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0
  0.0    0.0   -0.0    0.0    0.0    0.0    0.0    0.0   -0.0    0.0   -0.0  -0.0    0.0   -0.0    0.0    0.0
  0.0   -0.0    0.0    0.0   -0.0   -0.0    0.0   -0.0    0.0   -0.0   -0.0   0.0   -0.0   -0.0   -0.0   -0.0
 -0.0   -0.0   -0.0   -0.0    0.0   -0.0   -0.0    0.0   -0.0    0.0   -0.0  -0.0   -0.0   -0.0   -0.0   -0.0
 -0.0   -0.0    0.0   -0.0   -0.0    0.0   -0.0   -0.0   -0.0   -0.0    0.0  -0.0    0.0    0.0   -0.0    0.0
  0.0   -0.0    0.0    0.0   -0.0   -0.0    0.0   -0.0    0.0   -0.0    0.0   0.0   -0.0    0.0   -0.0   -0.0
  0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0
  0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0
  0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0

In [23]:
println("\nReconstruction of U=ΦA\n")
Urecons=Φ*A
show(stdout,"text/plain", round.(Urecons;digits=2))


Reconstruction of U=ΦA

10×16 Array{Float64,2}:
 -11.56   -8.67   -6.93   -5.78   -9.63   -7.22   -5.78   -4.81   -8.25   -6.19   -4.95   -4.13   -7.22   -5.42   -4.33   -3.61
 -28.53  -21.39  -17.12  -14.26  -23.77  -17.83  -14.26  -11.89  -20.38  -15.28  -12.23  -10.19  -17.83  -13.37  -10.7    -8.91
  -0.89   -0.67   -0.53   -0.44   -0.74   -0.56   -0.44   -0.37   -0.63   -0.48   -0.38   -0.32   -0.56   -0.42   -0.33   -0.28
 -31.87  -23.9   -19.12  -15.93  -26.55  -19.92  -15.93  -13.28  -22.76  -17.07  -13.66  -11.38  -19.92  -14.94  -11.95   -9.96
   7.11    5.33    4.27    3.56    5.93    4.44    3.56    2.96    5.08    3.81    3.05    2.54    4.44    3.33    2.67    2.22
 -16.54  -12.4    -9.92   -8.27  -13.78  -10.34   -8.27   -6.89  -11.81   -8.86   -7.09   -5.91  -10.34   -7.75   -6.2    -5.17
  -2.67   -2.0    -1.6    -1.33   -2.22   -1.67   -1.33   -1.11   -1.9    -1.43   -1.14   -0.95   -1.67   -1.25   -1.0    -0.83
 -35.53  -26.65  -21.32  -17.76  -29.61  -22.21  -17.76

In [24]:
println("\nReduction of Uapprox=Φ10*A10'~U\n")
Uapprox=Φ[:,10]*A[10,:]'
show(stdout,"text/plain", round.(Uapprox;digits=2))


Reduction of Uapprox=Φ10*A10'~U

10×16 Array{Float64,2}:
 -11.56   -8.67   -6.93   -5.78   -9.63   -7.22   -5.78   -4.81   -8.25   -6.19   -4.95   -4.13   -7.22   -5.42   -4.33   -3.61
 -28.53  -21.39  -17.12  -14.26  -23.77  -17.83  -14.26  -11.89  -20.38  -15.28  -12.23  -10.19  -17.83  -13.37  -10.7    -8.91
  -0.89   -0.67   -0.53   -0.44   -0.74   -0.56   -0.44   -0.37   -0.63   -0.48   -0.38   -0.32   -0.56   -0.42   -0.33   -0.28
 -31.87  -23.9   -19.12  -15.93  -26.55  -19.92  -15.93  -13.28  -22.76  -17.07  -13.66  -11.38  -19.92  -14.94  -11.95   -9.96
   7.11    5.33    4.27    3.56    5.93    4.44    3.56    2.96    5.08    3.81    3.05    2.54    4.44    3.33    2.67    2.22
 -16.54  -12.4    -9.92   -8.27  -13.78  -10.34   -8.27   -6.89  -11.81   -8.86   -7.09   -5.91  -10.34   -7.75   -6.2    -5.17
  -2.67   -2.0    -1.6    -1.33   -2.22   -1.67   -1.33   -1.11   -1.9    -1.43   -1.14   -0.95   -1.67   -1.25   -1.0    -0.83
 -35.53  -26.65  -21.32  -17.76  -29.61  -22.2

In [8]:
#Calculate the SVD
Z=svd(U)
println("\nSVD decomposition=\r")
display(Z)
println("\nUapprox based on the first energetic mode=\r")
Ũ=Z.U[:,1]*(diagm(Z.S)*Z.Vt)[1,:]'
println("\nŨ=\r")
display(Ũ)


EigenValues λ



10×10 Array{Float64,2}:
 -0.810758    0.452581    0.223793    0.0261588   …  -0.0826759   -0.175013
  0.330967    0.500254   -0.30218    -0.0220724      -0.204094    -0.432037
 -0.0868975  -0.030026   -0.0159381   0.129063       -0.00635969  -0.0134625
 -0.0560169  -0.0153869  -0.237248   -0.144207       -0.22799     -0.482621
 -0.381459   -0.305813   -0.631437   -0.526336        0.0508775    0.1077
 -0.0214691  -0.519928    0.305774   -0.0478099   …  -0.118332    -0.250492
 -0.195343   -0.220599   -0.453963    0.824502       -0.0190791   -0.0403875
 -0.0159586  -0.345177    0.168952   -0.00611412     -0.2542      -0.538104
  0.195084    0.110088   -0.274272   -0.0470995      -0.0190791   -0.0403875
  0.0         0.0         0.0         0.0             0.904187    -0.427138

[-1.6658361335067054e-13, -5.462803647822355e-14, -1.817489391522599e-14, -1.009819933969701e-14, 4.710919964930092e-16, 3.999910246483101e-13, 5.244053535157974e-13, 5.733961310215929e-13, 1.8189894035458565e-11, 24580.216532550156]

EigenVectors Matrix Φ



10×16 Array{Float64,2}:
 -1.3126e-15    2.17841e-15   2.92932e-15  …   1.0892e-15    9.62755e-16
  4.43051e-16   2.30498e-15  -6.16363e-15      1.15249e-15   5.35504e-16
  8.03026e-16  -5.15014e-15   6.18707e-15     -2.57507e-15  -2.00945e-15
 -1.89314e-15  -1.66932e-16  -2.4733e-15      -8.34661e-17  -4.25602e-16
 -1.17874e-15  -9.14403e-16   2.02445e-15     -4.57201e-16   1.22044e-15
  8.03238e-15  -3.99128e-15   5.47588e-15  …  -1.99564e-15  -7.54953e-16
  1.99277e-14   1.18661e-14   1.17257e-14      5.93303e-15   4.41721e-15
  9.01226e-16   9.5313e-15    7.71073e-16      4.76565e-15   2.87085e-15
  2.28362e-14   9.02195e-15   1.60673e-14      4.51098e-15   3.21137e-15
 66.027        49.5202       39.6162          24.7601       20.6334

10×16 Array{Float64,2}:
 -11.5556     -8.66667    -6.93333   …   -5.41667    -4.33333    -3.61111
 -28.5261    -21.3946    -17.1157       -13.3716    -10.6973     -8.91441
  -0.888889   -0.666667   -0.533333      -0.416667   -0.333333   -0.277778
 -31.866     -23.8995    -19.1196       -14.9372    -11.9497     -9.95812
   7.11111     5.33333     4.26667        3.33333     2.66667     2.22222
 -16.5392    -12.4044     -9.92352   …   -7.75275    -6.2022     -5.1685
  -2.66667    -2.0        -1.6           -1.25       -1.0        -0.833333
 -35.5294    -26.647     -21.3176       -16.6544    -13.3235    -11.1029
  -2.66667    -2.0        -1.6           -1.25       -1.0        -0.833333
 -28.2026    -21.1519    -16.9216       -13.22      -10.576      -8.81331

...


Amplitude Matrix A


Reconstruction of U=ΦA


Reduction of Uapprox=Φ10*A10'~U


SVD{Float64,Float64,Array{Float64,2}}
U factor:
10×10 Array{Float64,2}:
  0.175013    0.0908838     0.660406   …  -0.484511   -0.216359    0.0322019
  0.432037   -0.768103      0.179112       0.0878363   0.091867   -0.0529522
  0.0134625  -0.0174021     0.115046       0.536019    0.332963   -0.160737
  0.482621    0.482509      0.395497       0.206786    0.169657   -0.221492
 -0.1077     -0.208478      0.0286675     -0.469527    0.602244   -0.0302001
  0.250492    0.028493     -0.140961   …  -0.261284    0.281913    0.179778
  0.0403875  -0.256343      0.021577      -0.0240329  -0.591306   -0.179876
  0.538104   -0.065883     -0.272334       0.13054    -0.0489795   0.0633951
  0.0403875  -0.000510842   0.170825       0.197072   -0.032079    0.922278
  0.427138    0.233049     -0.487468      -0.286839   -0.0992865   0.0325882
singular values:
10-element Array{Float64,1}:
 156.78079133793835
   2.5477088009806055e-14
   1.3824020994007164e-14
   9.643292276369032e-15
   6.598096259151781



SVD decomposition=


10×16 Array{Float64,2}:
 -11.5556     -8.66667    -6.93333   …   -5.41667    -4.33333    -3.61111
 -28.5261    -21.3946    -17.1157       -13.3716    -10.6973     -8.91441
  -0.888889   -0.666667   -0.533333      -0.416667   -0.333333   -0.277778
 -31.866     -23.8995    -19.1196       -14.9372    -11.9497     -9.95812
   7.11111     5.33333     4.26667        3.33333     2.66667     2.22222
 -16.5392    -12.4044     -9.92352   …   -7.75275    -6.2022     -5.1685
  -2.66667    -2.0        -1.6           -1.25       -1.0        -0.833333
 -35.5294    -26.647     -21.3176       -16.6544    -13.3235    -11.1029
  -2.66667    -2.0        -1.6           -1.25       -1.0        -0.833333
 -28.2026    -21.1519    -16.9216       -13.22      -10.576      -8.81331


Uapprox based on the first energetic mode=

Ũ=
